In [1]:
import process as process

In [2]:
from parselmouth.praat import run_file
import os.path, operator
import glob                     # glob.glob
import json                     # json.load
import csv                      # csv.writer

In [3]:
cur_dir_path = os.path.dirname(os.path.realpath("process.py"))
cur_dir_path

'/Users/WYIQ/Downloads/result'

In [4]:
praat_path = cur_dir_path + "/myspsolution.praat"
textgrid_out_dir = "/Volumes/Backup Station/FYP/1986_1993/out/tmp"
splitted_mp3_dir = "/Volumes/Backup Station/FYP/1986_1993/out"
meta_json_dir = cur_dir_path + "/1986_1993/"
csv_out_path = cur_dir_path + "/result.csv"

In [5]:
process.obtain_meta_jsons(meta_json_dir)

{'/Users/WYIQ/Downloads/result/1986_1993/1989.88-6873.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1986.85-637.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1990.89-5867.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1986.86-80.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1986.85-1088.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1990.89-7370.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1993.93-518.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1991.90-1141.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1986.86-251.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1992.91-2051.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1989.88-5050.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1991.91-119.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1991.91-636.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1991.91-810.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1987.86-7059.json',
 '/Users/WYIQ/Downloads/result/1986_1993/1992.91_522.json',
 '/Users/WYIQ/Downloads/result/19

In [ ]:
process.myspf0sd("/Users/WYIQ/Downloads/result/out/1989.88-1105-t01/joseph_m_goldhammer.mp3")

In [ ]:
splitted_mp3_dir

In [6]:
process.get_filename_without_ext("/Users/WYIQ/Downloads/result/1986_1993/1990.89-1500.json")

'1990.89-1500'

In [7]:
process.process1('/Users/WYIQ/Downloads/result/1986_1993/1989.89-645.json')

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1989.89-645.json


['1989.89-645', -1, -1]

In [8]:
STR_APPELLANT = "appellant"
STR_APPELLEE = "appellee"
STR_UNKNOWN = "unknown"
CSV_HEADERS = ['ID', 'Petitioner f0_std', 'Respondents f0_std']

In [9]:
def myspf0sd(sound_path):
    try:
        objects = run_file(praat_path, -20, 2, 0.3, "yes",sound_path, textgrid_out_dir, 80, 400, 0.01, capture_output=True)
        # print (objects[0]) # This will print the info from the sound object, and objects[0] is a parselmouth.Sound object
        z1 = str(objects[1]) # This will print the info from the textgrid object, and objects[1] is a parselmouth.Data object with a TextGrid inside
        z2 = z1.strip().split()
        z3 = int(z2[3]) # will be the integer number 10
        z4 = float(z2[7]) # will be the floating point number 8.3
        # print ("f0_SD=",z4,"# Hz global standard deviation of fundamental frequency distribution")
    except:
        z4 = 0
        print ("[TODO] The sound of the audio was not clear:", sound_path)
        raise
    return z4

In [10]:
def process1(json_file):
    print ('Prosing file: ', json_file)
    advocateMap = {'brent_lawson_english': 'appellant', 'richard_d_panza': 'appellee'}
    # print('Obtain advocate map: ', advocateMap)

    filename_without_ext = '1989.89-645'
    mp3_out_dir = os.path.join(splitted_mp3_dir, filename_without_ext + "-t01")

    # list the files and get the largest two files
    all_files = (os.path.join(basedir, filename) for basedir, dirs, files in os.walk(mp3_out_dir) for filename in files)
    files_and_sizes = ((path, os.path.getsize(path)) for path in all_files)
    sorted_files_with_size = sorted(files_and_sizes, key = operator.itemgetter(1))
    sorted_files_with_size.reverse()
    top2_mp3_files = [item[0] for item in sorted_files_with_size[0:2]]
    csv_out = [filename_without_ext, -1, -1]
    for mp3_file in top2_mp3_files:
        std = myspf0sd(mp3_file)
        id = get_filename_without_ext(mp3_file)
        print ('{0:20s} : {1:9s} : std = {2}'.format(id, advocateMap[id], std))
        if advocateMap[id] == STR_APPELLANT:
            csv_out[1] = std
        elif advocateMap[id] == STR_APPELLEE:
            csv_out[2] = std
        else:
            print ('[Error] Advocate ', id, 'not exist in json file:', json_file)
            raise
    return csv_out

In [11]:
def get_filename_without_ext(fullpath):
    filename = fullpath.split(os.path.sep)[-1]
    filename_without_ext = os.path.splitext(filename)[0]
    return filename_without_ext

In [12]:
process.get_filename_without_ext('/Users/WYIQ/Downloads/result/1986_1993/1989.89-645.json')

'1989.89-645'

In [13]:
process.get_advocate_map('/Users/WYIQ/Downloads/result/1986_1993/1989.89-645.json')

{'brent_lawson_english': 'appellant', 'richard_d_panza': 'appellee'}

In [14]:
process1('/Users/WYIQ/Downloads/result/1986_1993/1989.89-645.json')

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1989.89-645.json
brent_lawson_english : appellant : std = 145.91
richard_d_panza      : appellee  : std = 142.83


['1989.89-645', 145.91, 142.83]

In [16]:
!python process.py

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1989.88-2041.json
warren_j_marwedel    : appellant : std = 123.31
robert_j_kopka       : appellee  : std = 120.43
Success

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1988.87-1855.json
martha_a_field       : appellee  : std = 167.36
maria_parisivickers  : appellee  : std = 218.16
Success

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1988.87-6325.json
Failed

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1991.90-6616.json
kenneth_j_rose       : appellant : std = 169.14
marvin_l_white_jr    : appellee  : std = 160.82
Success

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1993.92-1911.json
howard_e_shapiro     : appellant : std = 132.2
christine_o_gregoire : appellee  : std = 240.5
Success

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1987.87-578.json
Failed

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/1987.87-367.json
Failed

Prosing file:  /Users/WYIQ/Downloads/result/1986_1993/19